In [1]:

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#

game = pd.read_csv('../Fantasy-Premier-League/data/2021-22/gws/merged_gw.csv')
pd.set_option("display.max_columns",None)
sns.set_theme() # To apply basic seaborn theme

In [9]:
# we have some data issues to fix first
fix =game[['GW','round','kickoff_time','was_home','team']].query("GW == 36").drop_duplicates().sort_values(by=['team'])

fix.groupby()

,GW,round,kickoff_time,was_home,team
22706,36,36,2022-05-12T18:45:00Z,False,Arsenal
22705,36,36,2022-05-08T13:00:00Z,True,Arsenal
22631,36,36,2022-05-07T14:00:00Z,False,Aston Villa
22632,36,36,2022-05-10T19:00:00Z,True,Aston Villa
22667,36,36,2022-05-07T14:00:00Z,True,Brentford
22646,36,36,2022-05-07T16:30:00Z,True,Brighton
22707,36,36,2022-05-07T14:00:00Z,True,Burnley
22682,36,36,2022-05-11T18:30:00Z,False,Chelsea
22681,36,36,2022-05-07T14:00:00Z,True,Chelsea
22639,36,36,2022-05-07T14:00:00Z,True,Crystal Palace


In [17]:
#the features I thought I built.

game.head()
column_game = game.columns
stats = game.describe()

In [18]:
column_game


Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW'],
      dtype='object')

In [26]:
# we now focus on teams info

team = game[['team','GW','kickoff_time','round', 'team_a_score', 'team_h_score','was_home']].drop_duplicates()
# dir(team)


In [27]:
# there is an potential to use goal difference and league position to determine team strenght that we might want to pass on to the model


def points(row):
    # for row wise computation

    if row <-0:
        result = 0
    elif row == 0:
        result = 1
    else:
        result = 3
    return result

def home_away(row):
    if row['was_home'] == True:   # this is always frustrating row[(row['was_home'] == True)]
        value =  row['Home']
    else:
        value =  row['Away']
    return value
    


def adder (el):

    score = pd.Series(el['team_h_score']-el['team_a_score']).apply(points) # so this will return a  pandas series, our evaluation returns a series. So we need to do our evaluation right over here
    score_a = pd.Series(el['team_a_score']-el['team_h_score']).apply(points) 
    combine = {'Home':score, 'Away':score_a,'was_home':el['was_home']}
    df = pd.DataFrame(combine).apply(home_away,axis=1) #.apply(home_away)

    # if (el['was_home'] == True):
        

    return df


team['Points'] = team.pipe(adder) #.apply(home_away) # this argument passes itself as an instance
        


In [28]:

# looks like there is some reseting work 

team['Tally_Points'] = team.sort_values(by=['team','GW']).groupby('team')['Points'].cumsum()

team


,team,GW,kickoff_time,round,team_a_score,team_h_score,was_home,Points,Tally_Points
0,Man Utd,1,2021-08-14T11:30:00Z,1,1,5,True,3,3
1,Aston Villa,1,2021-08-14T14:00:00Z,1,2,3,False,0,0
2,Watford,1,2021-08-14T14:00:00Z,1,2,3,True,3,3
3,Southampton,1,2021-08-14T14:00:00Z,1,1,3,False,0,0
5,Liverpool,1,2021-08-14T16:30:00Z,1,3,0,False,3,3
...,...,...,...,...,...,...,...,...,...
23860,Chelsea,38,2022-05-22T15:00:00Z,38,1,2,True,3,73
23871,Spurs,38,2022-05-22T15:00:00Z,38,5,0,False,3,68
23875,Arsenal,38,2022-05-22T15:00:00Z,38,1,5,True,3,69
23876,Burnley,38,2022-05-22T15:00:00Z,38,2,1,True,0,34


In [29]:
team['team'].unique()
team

,team,GW,kickoff_time,round,team_a_score,team_h_score,was_home,Points,Tally_Points
0,Man Utd,1,2021-08-14T11:30:00Z,1,1,5,True,3,3
1,Aston Villa,1,2021-08-14T14:00:00Z,1,2,3,False,0,0
2,Watford,1,2021-08-14T14:00:00Z,1,2,3,True,3,3
3,Southampton,1,2021-08-14T14:00:00Z,1,1,3,False,0,0
5,Liverpool,1,2021-08-14T16:30:00Z,1,3,0,False,3,3
...,...,...,...,...,...,...,...,...,...
23860,Chelsea,38,2022-05-22T15:00:00Z,38,1,2,True,3,73
23871,Spurs,38,2022-05-22T15:00:00Z,38,5,0,False,3,68
23875,Arsenal,38,2022-05-22T15:00:00Z,38,1,5,True,3,69
23876,Burnley,38,2022-05-22T15:00:00Z,38,2,1,True,0,34


In [23]:
#here we need to figure out who is the opposing team


opps_df = game[['team','GW','round', 'team_a_score', 'team_h_score','was_home']].drop_duplicates()
opps_df

,team,GW,round,team_a_score,team_h_score,was_home
0,Man Utd,1,1,1,5,True
1,Aston Villa,1,1,2,3,False
2,Watford,1,1,2,3,True
3,Southampton,1,1,1,3,False
5,Liverpool,1,1,3,0,False
...,...,...,...,...,...,...
23860,Chelsea,38,38,1,2,True
23871,Spurs,38,38,5,0,False
23875,Arsenal,38,38,1,5,True
23876,Burnley,38,38,2,1,True


In [194]:
#let self merge first

pd.merge(opps_df
            ,opps_df
            ,how ='inner'
            ,left_on=['round','GW','team_a_score','team_h_score']
            ,right_on=['round','GW','team_h_score','team_a_score'])

# this g

,team_x,GW,round,team_a_score_x,team_h_score_x,was_home_x,team_y,team_a_score_y,team_h_score_y,was_home_y
0,Liverpool,1,1,3,0,False,Crystal Palace,0,3,False
1,Liverpool,1,1,3,0,False,Chelsea,0,3,True
2,Norwich,1,1,3,0,True,Crystal Palace,0,3,False
3,Norwich,1,1,3,0,True,Chelsea,0,3,True
4,Crystal Palace,1,1,0,3,False,Liverpool,3,0,False
...,...,...,...,...,...,...,...,...,...,...
1044,Brentford,38,38,2,1,True,Chelsea,1,2,True
1045,Leeds,38,38,2,1,False,Watford,1,2,False
1046,Leeds,38,38,2,1,False,Chelsea,1,2,True
1047,Burnley,38,38,2,1,True,Watford,1,2,False


In [195]:
# Self join to get the major attributes
T1 = pd.merge(game
        ,team.drop(['team_a_score','team_h_score','was_home','Points','GW'], axis=1) # drop these fields that we do not need
        ,how='left'
        ,left_on=['team','round']
        ,right_on=['team','round']
        ,suffixes=['','_t']
        )
T1 # we need to figure out a way to find the opposing team

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,Tally_Points
0,Eric Bailly,DEF,Man Utd,0.0,0,0,0,0,0.0,286,6,0,0,0.0,0.0,2021-08-14T11:30:00Z,0,10,0,0,0,0,1,0,9363,1,5,0.0,0,0,0,0,50,True,0,1,3
1,Keinan Davis,FWD,Aston Villa,0.4,0,0,0,0,0.0,49,8,0,0,0.0,0.0,2021-08-14T14:00:00Z,0,18,0,0,0,0,1,0,169789,2,3,0.0,0,0,0,0,45,False,0,1,0
2,Ayotomiwa Dele-Bashiru,MID,Watford,0.0,0,0,0,0,0.0,394,8,0,0,0.0,0.0,2021-08-14T14:00:00Z,0,2,0,0,0,0,1,0,4092,2,3,0.0,0,0,0,0,45,True,0,1,3
3,James Ward-Prowse,MID,Southampton,2.3,0,0,20,0,30.5,341,4,3,0,5.2,21.6,2021-08-14T14:00:00Z,90,8,0,0,0,0,1,0,299682,1,3,0.0,2,0,0,0,65,False,0,1,0
4,Bruno Miguel Borges Fernandes,MID,Man Utd,4.4,0,3,61,0,35.9,277,6,1,3,20.1,106.2,2021-08-14T11:30:00Z,90,10,0,0,0,0,1,0,3381004,1,5,59.0,20,0,0,0,120,True,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28186,Wilfred Ndidi,MID,Leicester,0.0,0,0,0,0,0.0,216,377,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,16,0,0,0,0,38,0,70366,1,4,0.0,0,-202,22,224,48,True,0,38,48
28187,Matt Ritchie,DEF,Newcastle,0.9,0,0,3,0,0.0,292,374,0,0,0.0,0.0,2022-05-22T15:00:00Z,1,5,0,0,0,0,38,0,15999,2,1,0.0,1,143,396,253,49,False,0,38,46
28188,Nathan Redmond,MID,Southampton,3.4,0,0,5,0,0.0,336,377,3,0,0.0,0.0,2022-05-22T15:00:00Z,82,9,0,0,0,0,38,0,13856,1,4,0.0,2,455,683,228,59,False,0,38,40
28189,Mathew Ryan,GK,Brighton,0.0,0,0,0,0,0.0,65,373,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,19,0,0,0,0,38,0,587,1,3,0.0,0,-2,0,2,45,True,0,38,49


In [204]:
# Self join the data to get 
T2 = (pd.merge(T1
                ,T1
                ,left_on=['name'] 
                ,right_on=['name']
                ,suffixes=("","_step")
                ).query("GW - GW_step == -1"))

                


In [205]:
#for readability


        


T2 =(T2
    .assign(was_home_ind =lambda x:T2['was_home'].replace({False: 0,True:1})) #previous game
    .assign(was_home_step_ind =lambda x:T2['was_home_step'].replace({False: 0,True:1})) # we will know next game info
    # .apply(playtime = lambda x:playtime(x))
    )
                

In [206]:

#adding one hot indicator for position
T2 = pd.get_dummies(data = T2,columns=["position"], prefix=["ind"])

                


In [207]:
# T1.minutes
def playtime(row):
        if row <5:
                result = 0
        else:
                result = 1
        return result

T2['playtime'] = T2['minutes'].apply(playtime) # need to figure out how to chain this.


In [208]:
T2[['name','team','team_step','Tally_Points_step','Tally_Points']]

,name,team,team_step,Tally_Points_step,Tally_Points
1,Eric Bailly,Man Utd,Man Utd,4,3
48,Eric Bailly,Man Utd,Man Utd,7,4
95,Eric Bailly,Man Utd,Man Utd,10,7
142,Eric Bailly,Man Utd,Man Utd,13,10
189,Eric Bailly,Man Utd,Man Utd,13,13
...,...,...,...,...,...
1135299,Alex Kirk,Arsenal,Arsenal,63,60
1135332,Matthew Craig,Spurs,Spurs,62,61
1135333,Matthew Craig,Spurs,Spurs,65,61
1135334,Matthew Craig,Spurs,Spurs,62,61


In [209]:
def team_diff(row):
    return row['Tally_Points_step']
    



In [30]:
team[(team['team']== 'Man City')]


,team,GW,kickoff_time,round,team_a_score,team_h_score,was_home,Points,Tally_Points
20,Man City,1,2021-08-15T15:30:00Z,1,0,1,False,0,0
575,Man City,2,2021-08-21T14:00:00Z,2,0,5,True,3,3
1142,Man City,3,2021-08-28T11:30:00Z,3,0,5,True,3,6
1719,Man City,4,2021-09-11T14:00:00Z,4,1,0,False,3,9
2318,Man City,5,2021-09-18T14:00:00Z,5,0,0,True,1,10
2924,Man City,6,2021-09-25T11:30:00Z,6,1,0,False,3,13
3535,Man City,7,2021-10-03T15:30:00Z,7,2,2,False,1,14
4148,Man City,8,2021-10-16T14:00:00Z,8,0,2,True,3,17
4765,Man City,9,2021-10-23T16:30:00Z,9,4,1,False,3,20
5383,Man City,10,2021-10-30T14:00:00Z,10,2,0,True,0,20


In [212]:
import arviz as az
T2.columns

Index(['name', 'team', 'xP', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW', 'Tally_Points',
       'position_step', 'team_step', 'xP_step', 'assists_step', 'bonus_step',
       'bps_step', 'clean_sheets_step', 'creativity_step', 'element_step',
       'fixture_step', 'goals_conceded_step', 'goals_scored_step',
       'ict_index_step', 'influence_step', 'kickoff_time_step', 'minutes_step',
       'opponent_team_step', 'own_goals_step', 'penalties_missed_step',
       'penalties_saved_step', 'red_cards_step', 'round_step', 'saves_step',
       'selected_step', 'team_a

In [220]:
# Isolate what we need from this dependant variables

depend = T2[[
            'GW'                                                    # Gameweek
            ,'playtime'                                             # this is an indicator function
            ,'xP'                                                   # this is an XP approach, i actually don't know what it means
            ,'transfers_balance', 'transfers_in', 'transfers_out'   # have to standardize this
            ,'ind_DEF', 'ind_FWD','ind_MID'                         #Position of players
            ,'was_home_ind'                                         # currently was this a home game?
            ,'was_home_step_ind'                                    # temporily this information is available to use
            ,'value'                                                # Current value of the player
            ,'total_points'                                         # Current performance of the player
            ,'Tally_Points'                                         # Points collected so far, need to rework this
            ,'total_points_step'                                    #
            ]]


explain = np.where(depend['total_points_step']<0,0,depend['total_points_step']) # we cant have -ve poisson count so we defualt players

In [222]:
# now I do not have a any transformations yet so lets do that!
depend.describe()

,playtime,xP,transfers_balance,transfers_in,transfers_out,ind_DEF,ind_FWD,ind_MID,was_home_ind,was_home_step_ind,value,total_points,Tally_Points,total_points_step
count,29486.000000,29486.000000,2.948600e+04,2.948600e+04,2.948600e+04,29486.000000,29486.000000,29486.000000,29486.000000,29486.000000,29486.000000,29486.000000,29486.000000,29486.000000
mean,0.396968,1.394350,3.092018e+03,1.575751e+04,1.266429e+04,0.340026,0.133318,0.411280,0.484806,0.502679,50.862715,1.215797,27.590585,1.216917
std,0.489278,2.305483,6.599258e+04,5.922963e+04,4.661307e+04,0.473726,0.339923,0.492074,0.499778,0.500001,12.533890,2.434907,17.264226,2.427323
min,0.000000,-2.000000,-1.857821e+06,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,37.000000,-4.000000,0.000000,-4.000000
25%,0.000000,0.000000,-9.220000e+02,3.800000e+01,1.050000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,44.000000,0.000000,15.000000,0.000000
50%,0.000000,0.500000,-2.800000e+01,3.990000e+02,9.190000e+02,0.000000,0.000000,0.000000,0.000000,1.000000,47.000000,0.000000,23.000000,0.000000
75%,1.000000,2.200000,1.547500e+02,5.413250e+03,7.213000e+03,1.000000,0.000000,1.000000,1.000000,1.000000,54.000000,1.000000,39.000000,2.000000
max,1.000000,21.600000,1.983733e+06,2.104464e+06,1.872898e+06,1.000000,1.000000,1.000000,1.000000,1.000000,133.000000,24.000000,83.000000,24.000000


In [219]:
#


AttributeError: 'numpy.ndarray' object has no attribute 'info'